In [1]:
import json

json_file_path = '/home/tinchung/Documents/GitHub/End-to-end-law-searching-with-RAG-Mamba/Law_corpus/law-corpus.json'

with open(json_file_path, 'rt') as f_in:
    docs_raw = json.load(f_in)
# docs_raw = [docs_raw]
docs_raw

documents = []

for law_title, law_data in docs_raw.items():
    law_title = law_data.get("law_title")
    law_number = law_data.get("law_number")
    chapters = law_data.get("chapters", [])
    
    for chapter in chapters:
        chapter_title = chapter.get("chapter_title")
        articles = chapter.get("articles", [])
        
        for article in articles:
            doc = {
                "law_title": law_title,
                "law_number": law_number,
                "chapter_title": chapter_title,
                "article_number": article.get("article_number"),
                "title": article.get("title"),
                "content": article.get("content")
            }
            documents.append(doc)

In [2]:
documents[10]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 11',
 'title': 'Ngôn ngữ, chữ viết dùng trong cơ sở giáo dục',
 'content': '1. Tiếng Việt là ngôn ngữ chính thức dùng trong cơ sở giáo dục. Căn cứ vào mục tiêu giáo dục và yêu cầu cụ thể về nội dung giáo dục, Chính phủ quy định việc dạy và học bằng tiếng nước ngoài trong cơ sở giáo dục. 2. Nhà nước khuyến khích, tạo điều kiện để người dân tộc thiểu số được học tiếng nói, chữ viết của dân tộc mình theo quy định của Chính phủ; người khuyết tật nghe, nói được học bằng ngôn ngữ ký hiệu, người khuyết tật nhìn được học bằng chữ nổi Braille theo quy định của Luật Người khuyết tật. 3. Ngoại ngữ quy định trong chương trình giáo dục là ngôn ngữ được sử dụng phổ biến trong giao dịch quốc tế. Việc tổ chức dạy ngoại ngữ trong cơ sở giáo dục phải bảo đảm để người học được học liên tục, hiệu quả. '}

In [3]:
import hashlib

def generate_document_id(doc):
    # combined = f"{doc['course']}-{doc['question']}"
    combined = f"{doc['law_title']}-{doc['chapter_title']}-{doc['title']}-{doc['content'][:10]}"
    hash_object = hashlib.md5(combined.encode())
    hash_hex = hash_object.hexdigest()
    document_id = hash_hex[:8]
    return document_id

In [4]:
for doc in documents:
    doc['id'] = generate_document_id(doc)

In [5]:
documents[3]

{'law_title': 'Luật Giáo Dục',
 'law_number': 'Luật số: 43/2019/QH14',
 'chapter_title': 'Chương I',
 'article_number': 'Điều 4',
 'title': 'Phát triển giáo dục',
 'content': '1. Phát triển giáo dục là quốc sách hàng đầu. 2. Phát triển giáo dục phải gắn với nhu cầu phát triển kinh tế - xã hội, tiến bộ khoa học, công nghệ, củng cố quốc phòng, an ninh; thực hiện chuẩn hóa, hiện đại hóa, xã hội hóa; bảo đảm cân đối cơ cấu ngành nghề, trình độ, nguồn nhân lực và phù hợp vùng miền; mở rộng quy mô trên cơ sở bảo đảm chất lượng và hiệu quả; kết hợp giữa đào tạo và sử dụng. 3. Phát triển hệ thống giáo dục mở, xây dựng xã hội học tập nhằm tạo cơ hội để mọi người được tiếp cận giáo dục, được học tập ở mọi trình độ, mọi hình thức, học tập suốt đời. ',
 'id': '06ce8ae6'}

In [6]:
from collections import defaultdict

In [7]:
hashes = defaultdict(list)

for doc in documents:
    doc_id = doc['id']
    hashes[doc_id].append(doc)

In [8]:
len(hashes), len(documents)

(314, 314)

In [9]:
for k, values in hashes.items():
    if len(values) > 1:
        print(k, len(values))

In [10]:
hashes['593f7569']

[]

In [11]:
import json
import os

In [12]:
output_dir = "./Law_corpus"
output_path = os.path.join(output_dir, 'documents-with-ids.json')

with open(output_path, 'wt') as f_out:
    json.dump(documents, f_out, ensure_ascii=False, indent=4)

In [13]:
!head ./Law_corpus/documents-with-ids.json

[
    {
        "law_title": "Luật Giáo Dục",
        "law_number": "Luật số: 43/2019/QH14",
        "chapter_title": "Chương I",
        "article_number": "Điều 1",
        "title": "Phạm vi điều chỉnh",
        "content": "Luật này quy định về hệ thống giáo dục quốc dân; cơ sở giáo dục, nhà giáo, người học; quản lý nhà nước về giáo dục; quyền và trách nhiệm của cơ quan, tổ chức, cá nhân liên quan đến hoạt động giáo dục. ",
        "id": "75a9286e"
    },


In [ ]:
prompt_template = """
You emulate a user who looking for law assistant.
Formulate 5 questions this user might ask based on a Vietnamese Law record. The record
should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

section: {section}
question: {question}
answer: {text}

Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [16]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv() 
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(
    api_key = OPENAI_API_KEY
)

In [17]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini-2024-07-18',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [18]:
from tqdm.auto import tqdm

In [19]:
results = {}

In [20]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions = generate_questions(doc)
    results[doc_id] = questions

  0%|          | 0/314 [00:00<?, ?it/s]

KeyError: 'section'

In [ ]:
import pickle

In [ ]:
with open('results.bin', 'rb') as f_in:
    results = pickle.load(f_in)

In [ ]:
results['1f6520ca']

'["Where can I find the prerequisites for this course?", "How do I check the prerequisites for this course?", "Where are the course prerequisites listed?", "What are the requirements for joining this course?", "Where is the list of prerequisites for the course?"]'

In [ ]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)

In [ ]:
doc_index = {d['id']: d for d in documents}

In [ ]:
final_results = []

for doc_id, questions in parsed_resulst.items():
    course = doc_index[doc_id]['course']
    for q in questions:
        final_results.append((q, course, doc_id))

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(final_results, columns=['question', 'course', 'document'])

In [ ]:
df.to_csv('ground-truth-data.csv', index=False)

In [ ]:
!head ground-truth-data.csv

question,course,document
When does the course begin?,data-engineering-zoomcamp,c02e79ef
How can I get the course schedule?,data-engineering-zoomcamp,c02e79ef
What is the link for course registration?,data-engineering-zoomcamp,c02e79ef
How can I receive course announcements?,data-engineering-zoomcamp,c02e79ef
Where do I join the Slack channel?,data-engineering-zoomcamp,c02e79ef
Where can I find the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
How do I check the prerequisites for this course?,data-engineering-zoomcamp,1f6520ca
Where are the course prerequisites listed?,data-engineering-zoomcamp,1f6520ca
What are the requirements for joining this course?,data-engineering-zoomcamp,1f6520ca
